In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import json

In [3]:
with open('kaggle .json', 'r') as f:
    data = json.load(f)

In [4]:
data

{'username': 'aabithmusthaq', 'key': 'b91cc2cd1adafc85b6ddd87b77fea8fc'}

In [5]:
!pip install kaggle


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import os
os.environ['KAGGLE_USERNAME'] = data['username']
os.environ['KAGGLE_KEY'] = data['key']

In [7]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [8]:
api.dataset_download_files('dineshpiyasamara/sentiment-analysis-dataset')

Dataset URL: https://www.kaggle.com/datasets/dineshpiyasamara/sentiment-analysis-dataset


In [9]:
import zipfile
with zipfile.ZipFile('sentiment-analysis-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [10]:
df = pd.read_csv(r"C:\Users\aabidh musthaq\Desktop\sentimenal project\review-sentimental-analysis\data\sentiment_analysis.csv")

In [11]:
df.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


## Data preprocessing

In [12]:
df.shape

(7920, 3)

In [13]:
df.duplicated().sum()

0

In [14]:
df.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

### Text preprocessing

In [15]:
import re #regular expression
import string # to remove punctuation

In [16]:
df['tweet'].head()

0    #fingerprint #Pregnancy Test https://goo.gl/h1...
1    Finally a transparant silicon case ^^ Thanks t...
2    We love this! Would you go? #talk #makememorie...
3    I'm wired I know I'm George I was made that wa...
4    What amazing service! Apple won't even talk to...
Name: tweet, dtype: object

In [17]:
#CONVER UPPER CASE TO LOWER CASE
df['tweet'] = df['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split())) #lower case conversion split the words and join them

In [18]:
df['tweet'].head()

0    #fingerprint #pregnancy test https://goo.gl/h1...
1    finally a transparant silicon case ^^ thanks t...
2    we love this! would you go? #talk #makememorie...
3    i'm wired i know i'm george i was made that wa...
4    what amazing service! apple won't even talk to...
Name: tweet, dtype: object

In [19]:
#remove links 
df["tweet"] = df['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split())) #remove links 
#multilines means it will remove the links in the entire data

In [20]:
df['tweet'].head()

0    #fingerprint #pregnancy test  #android #apps #...
1    finally a transparant silicon case ^^ thanks t...
2    we love this! would you go? #talk #makememorie...
3    i'm wired i know i'm george i was made that wa...
4    what amazing service! apple won't even talk to...
Name: tweet, dtype: object

remove puncuation

In [21]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [22]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [23]:
df["tweet"] = df["tweet"].apply(remove_punctuations)

In [24]:
df['tweet'].head()

0    fingerprint pregnancy test  android apps beaut...
1    finally a transparant silicon case  thanks to ...
2    we love this would you go talk makememories un...
3    im wired i know im george i was made that way ...
4    what amazing service apple wont even talk to m...
Name: tweet, dtype: object

In [25]:
df["tweet"] = df['tweet'].str.replace('\d+', '', regex=True) #remove numbers

In [26]:
import nltk

In [27]:
nltk.download('stopwords', download_dir='../static/model') #download stopwords ex: is, am, are, the, etc

[nltk_data] Downloading package stopwords to ../static/model...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
with open('../static/model/corpora/stopwords/english', 'r') as file: #open the file
    sw = file.read().splitlines() #read the stopwords and split them

In [29]:
df["tweet"] = df["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw)) #remove stopwords

In [30]:
from nltk.stem import PorterStemmer #stemming is the process of reducing a word to its word stem
ps = PorterStemmer() #stemming

In [31]:
df["tweet"] = df["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split())) #stemming ex: running -> run

### Vocablary buliding 

In [35]:
from collections import Counter #vocablary bilding 
vocab = Counter()

In [36]:
vocab

Counter()

In [38]:
for sentence in df['tweet']: #going throuhgt each word and count
    vocab.update(sentence.split()) 

In [39]:
vocab

Counter({'fingerprint': 5,
         'pregnanc': 1,
         'test': 12,
         'android': 414,
         'app': 416,
         'beauti': 322,
         'cute': 322,
         'health': 120,
         'iger': 143,
         'iphoneonli': 129,
         'iphonesia': 137,
         'iphon': 4073,
         'final': 192,
         'transpar': 5,
         'silicon': 5,
         'case': 348,
         'thank': 263,
         'uncl': 4,
         'yay': 69,
         'soni': 819,
         'xperia': 60,
         'sonyexperias…': 1,
         'love': 499,
         'would': 146,
         'go': 209,
         'talk': 27,
         'makememori': 1,
         'unplug': 3,
         'relax': 41,
         'smartphon': 134,
         'wifi': 50,
         'connect': 27,
         'im': 294,
         'wire': 8,
         'know': 106,
         'georg': 1,
         'made': 72,
         'way': 53,
         'daventri': 1,
         'home': 143,
         'amaz': 178,
         'servic': 51,
         'appl': 2876,
         'wont':

In [40]:
len(vocab)

15949

In [41]:
df.shape #rows low column hight overfit we can use corealtion to select feature

(7920, 3)

In [46]:
tokens = [key for key in vocab if vocab[key] > 15] #only keep which vocab is above 20 

In [47]:
len(tokens)

831

In [48]:
def save_vocabulary(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w', encoding="utf-8")
    file.write(data)
    file.close()

save_vocabulary(tokens, '../static/model/vocabulary.txt')

## Divide dataset